In [1]:
import requests
import pandas as pd
import sys
import random


In [21]:
df = pd.read_csv('/Users/admlanno/Desktop/projects/EbitdaAnalysis/data/df_main.csv')

In [22]:

def safe_convert(value):
    try:
        return float(value) if value is not None else 0
    except ValueError:
        return 0

def calculate_ebitda(inn):
    session = requests.Session()

    headers = {
        "Accept": "*/*",
        "Accept-Encoding": "gzip, deflate, br, zstd",
        "Accept-Language": "en-GB,en;q=0.9",
        "Cache-Control": "no-cache",
        "Connection": "keep-alive",
        "Cookie": "disclaimed=true",
        "Host": "bo.nalog.ru",
        "Pragma": "no-cache",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36 Edg/126.0.0.0",
        "sec-ch-ua": '"Not/A)Brand";v="8", "Chromium";v="126", "Microsoft Edge";v="126"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "Windows",
    }

    referers = [
        "https://bo.nalog.ru/",
        "https://bo.nalog.ru/otherpage",
        "https://bo.nalog.ru/anotherpage"
    ]

    headers["Referer"] = random.choice(referers)

    try:
        res = session.get(f"https://bo.nalog.ru/advanced-search/organizations/search?query={inn}&page=0", headers=headers).json()
    except requests.exceptions.JSONDecodeError:
        return None, "Ошибка декодирования JSON - сервер вернул невалидный ответ"
    except requests.exceptions.RequestException as e:
        return None, f"Ошибка запроса: {e}"

    if not res.get('content'):
        return None, "Организация не найдена"

    org_id = res['content'][0]['id']

    headers["Referer"] = random.choice(referers)

    try:
        financial_data = session.get(f"https://bo.nalog.ru/nbo/organizations/{org_id}/bfo", headers=headers).json()
    except requests.exceptions.JSONDecodeError:
        return None, "Ошибка декодирования JSON - сервер вернул невалидный ответ"
    except requests.exceptions.RequestException as e:
        return None, f"Ошибка запроса: {e}"

    ebitda_list = []

    for year_data in financial_data:
        financial_result = year_data.get('financialResult', {})
        balance = year_data.get('correction', {}).get('balance', {})

        profit = safe_convert(financial_result.get('current2400'))
        taxes = safe_convert(financial_result.get('current2410'))
        interest = safe_convert(financial_result.get('current2350'))
        depreciation = safe_convert(balance.get('current1150'))

        ebitda = profit + taxes + interest + depreciation
        ebitda_list.append({
            'year': year_data['period'],
            'ebitda': ebitda
        })

    return ebitda_list, None

# Инициализация переменных
results = []
total_inns = len(df)
checked_inns = 0

# Основной цикл обработки ИНН
for index, row in df.iterrows():
    inn = row['ИНН']
    ebitda_data, error = calculate_ebitda(inn)

    # Увеличиваем счетчик
    checked_inns += 1

    # Обновляем динамический счетчик на одной строке
    sys.stdout.write(f"\rПроверено {checked_inns} из {total_inns} ИНН.")
    sys.stdout.flush()

    if error:
        # Пропускаем вывод ошибок, но можем записать их в лог, если нужно
        continue

    # Добавляем данные в результирующий список
    for data in ebitda_data:
        results.append({
            'Наименование': row['Наименование / ФИО'],
            'ИНН': inn,
            'Год': data['year'],
            'EBITDA': data['ebitda']
        })
        



# Преобразуем результаты в DataFrame
ebitda_df = pd.DataFrame(results)


Проверено 142930 из 142930 ИНН.

In [23]:
ebitda_df = pd.DataFrame(results)

In [24]:
ebitda_df.shape

(484972, 4)

In [28]:
ebitda_df.to_csv('EBITDA.csv', index=False)

In [6]:
import numpy as np

# Задаем матрицу
matrix = np.array([[1/2, 1/2],
                   [1/3, 2/3]])


eigenvalues, eigenvectors = np.linalg.eig(matrix)

eigenvalues, eigenvectors


(array([0.16666667, 1.        ]),
 array([[-0.83205029, -0.70710678],
        [ 0.5547002 , -0.70710678]]))